## Problem Statement
Use any dataset of plant disease and design a plant disease detection system
using CNN.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [2]:
train_dir = "custom/train"
val_dir = "custom/valid"
test_dir = "custom/test"

In [3]:
img_size = 224
batch_size = 32

## Preprocessing

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_size, img_size),
                                                    batch_size=batch_size, 
                                                    class_mode='categorical')

Found 7316 images belonging to 4 classes.


In [5]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical')

Found 1829 images belonging to 4 classes.


## Taking **corn** class

In [6]:
from pprint import pprint

pprint(list(train_generator.class_indices), compact=True)

['Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy']


## Building Model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [8]:
model = Sequential()
model.add((Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3))))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(128, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Flatten()))
model.add((Dense(128, activation='relu')))
model.add((Dropout(0.2)))
model.add((Dense(64, activation='relu')))
model.add((Dense(train_generator.num_classes, activation='softmax')))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      256       
 chNormalization)                                                
                                                      

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Traning Our Model

In [10]:
model.fit(train_generator, epochs=50, validation_data=val_generator)

Epoch 1/50


229/229 [==============================] - 126s 543ms/step - loss: 0.4069 - accuracy: 0.8969 - val_loss: 7.1070 - val_accuracy: 0.3160
Epoch 2/50
229/229 [==============================] - 97s 422ms/step - loss: 0.1955 - accuracy: 0.9404 - val_loss: 0.8356 - val_accuracy: 0.8179
Epoch 3/50
229/229 [==============================] - 98s 429ms/step - loss: 0.1469 - accuracy: 0.9511 - val_loss: 0.1639 - val_accuracy: 0.9420
Epoch 4/50
229/229 [==============================] - 98s 427ms/step - loss: 0.0970 - accuracy: 0.9675 - val_loss: 0.1386 - val_accuracy: 0.9623
Epoch 5/50
229/229 [==============================] - 101s 440ms/step - loss: 0.0811 - accuracy: 0.9717 - val_loss: 0.3400 - val_accuracy: 0.9273
Epoch 6/50
229/229 [==============================] - 96s 421ms/step - loss: 0.0937 - accuracy: 0.9679 - val_loss: 1.6981 - val_accuracy: 0.7485
Epoch 7/50
229/229 [==============================] - 100s 438ms/step - loss: 0.0834 - accuracy: 0.9717 - val_loss: 0.3867 - v

### Evalulating Our Model

In [11]:
loss, accuracy = model.evaluate(val_generator)
print("Loss :", loss)
print("Accuracy (Test Data) :",accuracy*100)

58/58 [==============================] - 7s 121ms/step - loss: 0.2535 - accuracy: 0.9623
Loss : 0.2535024881362915
Accuracy (Test Data) : 96.22744917869568


## Testing The Model

In [23]:
class_names= [
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight',
    'Corn_(maize)___healthy']

In [24]:
import numpy as np
import os

test_imgs = os.listdir('custom/test')
pprint(test_imgs)

['CornCommonRust1.JPG', 'CornCommonRust2.JPG', 'CornCommonRust3.JPG']


In [25]:
for test_img in test_imgs:
    print('\n')
    img_path = 'custom/test/' + test_img
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    predicted_class = np.argmax(prediction)
    print(test_img, 'prediction:', prediction)
    print(test_img, 'predicted class: ', predicted_class)
    print('Predicted class:', class_names[predicted_class])



CornCommonRust1.JPG prediction: [[0. 1. 0. 0.]]
CornCommonRust1.JPG predicted class:  1
Predicted class: Corn_(maize)___Common_rust_


CornCommonRust2.JPG prediction: [[0. 1. 0. 0.]]
CornCommonRust2.JPG predicted class:  1
Predicted class: Corn_(maize)___Common_rust_


CornCommonRust3.JPG prediction: [[0. 1. 0. 0.]]
CornCommonRust3.JPG predicted class:  1
Predicted class: Corn_(maize)___Common_rust_
